# Going Modular

* This note book consists of cell mode code.
* The notebook 06.2 Going Modular consists of the python scripts to run the model in terminal using a single command.

In [1]:
import torch
device='cuda' if torch.cuda.is_available() else 'cpu'
print(f'Device: {device}')

Device: cpu


# Get the data

In [2]:
#Import necessary libraries
import os
import zipfile
from pathlib import Path
import requests

#Define paths
data_path=Path('data/')
image_path=data_path/'pizza_sushi_steak'

#Create the image folder if sdoes not exists
if image_path.is_dir():
  print(f"Directory {image_path} already exixts!")
else:
  print("Did not find image directory .Creating the directory....")
  image_path.mkdir(parents=True,exist_ok=True)
  
#Get the image data from github and write
with open(data_path/'pizza_sushi_steak.zip','wb') as f:
  request=requests.get('https://github.com/HarshEkbote/PyTorch-Basics/raw/main/data/pizza_steak_sushi.zip')
  f.write(request.content)
  print('Data Downloaded')

#Unzip the file contents
with zipfile.ZipFile(data_path/'pizza_sushi_steak.zip','r') as zip_ref:
  zip_ref.extractall(image_path)
  print('File Unzipped!!')

#Remove the zipfile
os.remove(data_path/'pizza_sushi_steak.zip')

Did not find image directory .Creating the directory....
Data Downloaded
File Unzipped!!


In [3]:
train_dir=image_path/'train'
test_dir=image_path/'test'

train_dir,test_dir

(WindowsPath('data/pizza_sushi_steak/train'),
 WindowsPath('data/pizza_sushi_steak/test'))

# Create the Dataset and DataLoader

In [4]:
from torchvision import datasets, transforms

data_transforms=transforms.Compose([
    transforms.Resize(size=(64,64)),
    transforms.ToTensor()
])

train_data=datasets.ImageFolder(root=train_dir,
                                transform=data_transforms,
                                target_transform=None)

test_data=datasets.ImageFolder(root=test_dir,
                                transform=data_transforms)

print(f"Train Datasets\n{train_data}\n\nTest Dataset\n{test_data}")

c:\Users\DELL\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Train Datasets
Dataset ImageFolder
    Number of datapoints: 225
    Root location: data\pizza_sushi_steak\train
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
           )

Test Dataset
Dataset ImageFolder
    Number of datapoints: 75
    Root location: data\pizza_sushi_steak\test
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
           )


In [5]:
class_name=train_data.classes
class_name

['pizza', 'steak', 'sushi']

In [6]:
class_dict=train_data.class_to_idx
class_dict

{'pizza': 0, 'steak': 1, 'sushi': 2}

In [7]:
len(train_data),len(test_data)

(225, 75)

In [8]:
from torch.utils.data import DataLoader
BATCH_SIZE=32

train_dataloader=DataLoader(
    dataset=train_data,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=1
)


test_dataloader=DataLoader(
    dataset=test_data,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=1
)

train_dataloader,test_dataloader

(<torch.utils.data.dataloader.DataLoader at 0x1f984c3f250>,
 <torch.utils.data.dataloader.DataLoader at 0x1f984c3eb30>)

In [9]:
img,label=next(iter(train_dataloader))
print(f"Image shape: {img.shape}\nLabel shape: {label.shape}")

Image shape: torch.Size([32, 3, 64, 64])
Label shape: torch.Size([32])


# Building the model -> TinyVgg

In [10]:
import torch
from torch import nn

class TinyVGG(nn.Module):
  def __init__(self,input_shape:int,hidden_units:int,output_shape:int):
    super().__init__()
    #Block 1
    self.block1=nn.Sequential(
        nn.Conv2d(in_channels=input_shape,out_channels=hidden_units,kernel_size=3,stride=1,padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,out_channels=hidden_units,kernel_size=3,padding=0,stride=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,stride=2)
    )
    #Block 2
    self.block2=nn.Sequential(
        nn.Conv2d(in_channels=hidden_units,out_channels=hidden_units,kernel_size=3,stride=1,padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,out_channels=hidden_units,kernel_size=3,padding=0,stride=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,stride=2)
    )
    #Classifier
    self.classifier=nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_units*13*13,out_features=output_shape)
    )

  def forward(self,x):
    # x=self.block1(x)
    # print(x.shape)
    # x=self.block2(x)
    # print(x.shape)
    # x=self.classifier(x)
    # print(x.shape)
    return self.classifier(self.block2(self.block1(x)))
    return x

In [11]:
torch.manual_seed(42)

#Initialize the model
model0=TinyVGG(input_shape=3,hidden_units=10,output_shape=len(class_name))
model0

TinyVGG(
  (block1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1690, out_features=3, bias=True)
  )
)

In [12]:
#Trial run to get the intermidiate size of tensor
img_batch, label_batch=next(iter(train_dataloader))
img_single,label_single=img_batch[0].unsqueeze(dim=0),label_batch[0]

model0.eval()
with torch.inference_mode():
  pred=model0(img_single.to(device))

print(f"Output logits:\n{pred}\n")
print(f"Output prediction probabilities:\n{torch.softmax(pred, dim=1)}\n")
print(f"Output prediction label:\n{torch.argmax(torch.softmax(pred, dim=1), dim=1)}\n")
print(f"Actual label:\n{label_single}")

Output logits:
tensor([[ 0.0208, -0.0020,  0.0095]])

Output prediction probabilities:
tensor([[0.3371, 0.3295, 0.3333]])

Output prediction label:
tensor([0])

Actual label:
0


# Creating the Training Loop

In [14]:
from typing import Tuple

def train_step(model:nn.Module,dataloader:torch.utils.data.DataLoader,loss_fn:torch.nn.Module,optimizer:torch.optim.Optimizer,device:torch.device)->Tuple[float,float]:
  model.train()

  train_loss,train_acc=0,0
  for batch,(x,y) in enumerate(dataloader):
    x,y=x.to(device),y.to(device)

    y_pred=model(x)

    loss=loss_fn(y_pred,y)
    train_loss+=loss.item()

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

    y_pred_class=torch.argmax(torch.softmax(y_pred,dim=1),dim=1)
    train_acc+=(y_pred_class==y).sum().item()/len(y_pred)

  train_loss=train_loss/len(dataloader)
  train_acc=train_acc/len(dataloader)
  return train_loss,train_acc

# Creating the Test Loop

In [15]:
def test_step(model:nn.Module,dataloader:torch.utils.data.DataLoader,loss_fn:torch.nn.Module,device:torch.device)->Tuple[float,float]:
  model.eval()
  test_loss,test_acc=0,0
  with torch.inference_mode():
    for batch,(x,y) in enumerate(dataloader):
      x,y=x.to(device),y.to(device)

      test_pred_logits=model(x)

      loss=loss_fn(test_pred_logits,y)
      test_loss+=loss.item()

      test_pred_label=test_pred_logits.argmax(dim=1)
      test_acc+=((test_pred_label==y).sum().item()/len(test_pred_label))

  test_loss=test_loss/len(dataloader)
  test_acc=test_acc/len(dataloader)

  return test_loss,test_acc

# Combinig the Train and Test step

In [16]:
from typing import Dict,List

from tqdm.auto import tqdm
def train(model:torch.nn.Module,train_datalaoder:torch.utils.data.DataLoader,
          test_dataloader:torch.utils.data.DataLoader,optimizer:torch.optim.Optimizer,loss_fn:torch.nn.Module,epochs:int,device:torch.device=device):
  
  results={
      'train_loss':[],
      'train_acc':[],
      'test_loss':[],
      'test_acc':[]
  }

  for epoch in tqdm(range(epochs)):
    train_loss,train_acc=train_step(model,train_dataloader,loss_fn,optimizer,device)
    test_loss,test_acc=test_step(model,test_dataloader,loss_fn,device)

    print(
        f"Epoch: {epoch+1}",
        f"train_loss:{train_loss:.4f} |"
        f"train_acc: {train_acc:.2f} |"
        f"test_loss:{test_loss:.4f} |"
        f"test_acc: {test_acc:.2f} |")
    
    results['train_loss'].append(train_loss)
    results['train_acc'].append(train_acc)
    results['test_loss'].append(test_loss)
    results['test_acc'].append(test_acc)
  return results


# Creating a Function to Save a model

In [18]:
from pathlib import Path
def save_model(model:torch.nn.Module,target_dir:str,model_name:str):
  target_dir_path=Path(target_dir)
  target_dir_path.mkdir(parents=True,exist_ok=True)

  assert model_name.endswith('.pth') or model_name.endswith('.pt'), 'model must be saved in .pth or .pt format'
  model_save_path=target_dir_path/model_name

  print(f"[INFO] Saving the mode to {model_save_path}")
  torch.save(obj=model.state_dict(),f=model_save_path)

# Train, Evaluate and Save the model

In [19]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

NUM_EPOCHS=5

model0=TinyVGG(input_shape=3,hidden_units=10,output_shape=len(class_name)).to(device)

loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(params=model0.parameters(),lr=0.01)

from timeit import default_timer as timer

start_time=timer()

model0_results=train(model=model0,
                     train_datalaoder=train_dataloader,
                     test_dataloader=test_dataloader,
                     optimizer=optimizer,
                     loss_fn=loss_fn,epochs=NUM_EPOCHS,device=device)

end_time=timer()
print(f'[INFO] Total training time: {end_time-start_time:.3f} seconds')


save_model(model=model0,target_dir='models',model_name="cell_mode_model.pth")


 20%|██        | 1/5 [00:11<00:47, 11.85s/it]

Epoch: 1 train_loss:1.2058 |train_acc: 0.34 |test_loss:1.1186 |test_acc: 0.54 |


 40%|████      | 2/5 [00:20<00:30, 10.13s/it]

Epoch: 2 train_loss:1.1685 |train_acc: 0.28 |test_loss:1.0783 |test_acc: 0.54 |


 60%|██████    | 3/5 [00:29<00:18,  9.29s/it]

Epoch: 3 train_loss:1.1069 |train_acc: 0.32 |test_loss:1.0955 |test_acc: 0.42 |


 80%|████████  | 4/5 [00:37<00:09,  9.05s/it]

Epoch: 4 train_loss:1.0997 |train_acc: 0.31 |test_loss:1.0937 |test_acc: 0.54 |


100%|██████████| 5/5 [00:45<00:00,  9.19s/it]

Epoch: 5 train_loss:1.0979 |train_acc: 0.40 |test_loss:1.0958 |test_acc: 0.54 |
[INFO] Total training time: 45.974 seconds
[INFO] Saving the mode to models\cell_mode_model.pth
